In [1]:
# imports
from importlib import reload

import os # a module to interact with the operating system
os.environ["ENDPOINT_URL"]="http://rook-ceph-rgw-nautiluss3.rook"
import numpy as np

import pandas as pd
import xarray
import h5py
import healpy as hp
import time

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.llc import plotting as llc_plotting

from ulmo.utils import image_utils
from ulmo.analysis import figures as ulmo_figs
from ulmo.plotting import plotting as ulmo_plotting
from ulmo.analysis import spatial_plots as sp

# Load table

In [2]:
viirs_tbl = ulmo_io.load_main_table( 's3://viirs/Tables/VIIRS_all_99clear_std.parquet' )
viirs_tbl.head()

Read main table: s3://viirs/Tables/VIIRS_all_99clear_std.parquet


,filename,row,col,lat,lon,clear_fraction,field_size,datetime,ex_filename,pp_file,pp_root,pp_idx,pp_type,mean_temperature,Tmin,Tmax,T90,T10,LL
0,s3://viirs/data/2012/032/20120201001000-OSPO-L...,3216,2496,21.009575,-175.17024,0.004693,192,2012-02-01 00:10:00,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,viirs_std,459126,0,26.149899,25.119995,26.790009,26.359985,25.950012,482.765015
1,s3://viirs/data/2012/032/20120201001000-OSPO-L...,3216,2592,20.876402,-175.80142,0.002740,192,2012-02-01 00:10:00,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,viirs_std,379958,0,26.039507,24.489990,26.889984,26.309998,25.829987,493.697144
2,s3://viirs/data/2012/032/20120201001000-OSPO-L...,3216,2688,20.725744,-176.50098,0.006565,192,2012-02-01 00:10:00,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,viirs_std,217941,0,25.925251,23.980011,27.029999,26.359985,25.609985,328.781921
3,s3://viirs/data/2012/032/20120201001000-OSPO-L...,3312,2496,21.622444,-175.36200,0.003906,192,2012-02-01 00:10:00,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,viirs_std,455088,0,26.135242,25.160004,27.149994,26.399994,25.920013,447.812378
4,s3://viirs/data/2012/032/20120201001000-OSPO-L...,3408,1920,23.276363,-170.16823,0.003988,192,2012-02-01 00:10:00,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,viirs_std,200120,0,26.035648,24.579987,27.320007,26.600006,25.359985,308.331177


# Create h5 File

In [3]:
file_list = pd.unique( viirs_tbl.pp_file)

In [4]:
file = file_list[8]
file

's3://viirs/PreProc/VIIRS_2020_95clear_192x192_preproc_viirs_std.h5'

In [5]:
sub_tbl = viirs_tbl[ file == viirs_tbl.pp_file ]
sub_tbl.shape[0]

265988

In [6]:
index_list = sub_tbl.pp_idx.sort_values()
index_list

2123523          5
2230199         21
2148127         22
2257757         27
2212166         29
            ...   
2250983    1149945
2290569    1149946
2104385    1149951
2273690    1149963
2094189    1149964
Name: pp_idx, Length: 265988, dtype: int64

In [7]:
chunk_list = np.arange(0, sub_tbl.shape[0], 10000 )
chunk_list.shape[0]

27

In [8]:
# initialize image matrix
cutouts = np.ndarray( (viirs_tbl.shape[0], 1, 64, 64) )

In [9]:
count = 231905 + 258998 + 247455 + 236728 + 251215 + 263844 + 255667 + 279791
count

2025603

In [10]:
for chunk in chunk_list: 
    
    if sub_tbl.shape[0] - chunk >= 10000: 
        
        # grab indices
        idxs = index_list[chunk: chunk + 10000].to_numpy()
        
        # grab cutouts
        with h5py.File('/home/jovyan/VIIRS_2020_95clear_192x192_preproc_viirs_std.h5') as f: 
            
            imgs = f['valid'][idxs, 0, ...]
            
        # save cutouts in an array
        marker = count + chunk  
        cutouts[marker: marker + 10000, 0, ...] = imgs
        
    else: 
        
        # grab indices
        idxs = index_list[chunk: sub_tbl.shape[0]].to_numpy()
        
        # grab cutouts
        with h5py.File('/home/jovyan/VIIRS_2020_95clear_192x192_preproc_viirs_std.h5') as f: 
            
            imgs = f['valid'][idxs, 0, ...]
            
        # save cutouts in an array
        marker = count + chunk 
        diff = sub_tbl.shape[0] - chunk
        cutouts[marker: marker + diff, 0, ...] = imgs
        
        count += sub_tbl.shape[0]

In [14]:
cutouts[count, 0, ...]

IndexError: index 2291591 is out of bounds for axis 0 with size 2291591

In [15]:
count

2291591

In [17]:
with h5py.File( '/home/jovyan/VIIRS_all_99clear_192x192_preproc.h5', 'r+') as g:
    
    g.create_dataset( name='2020', data = cutouts[2025603:2291591, 0, ...])